# 준비

In [25]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
%cd /content/drive/MyDrive/Project_Music_KW

/content/drive/MyDrive/Project_Music_KW


In [27]:
import glob
import pickle
import numpy as np
from music21 import converter, instrument, note, chord
from keras import optimizers
from keras import models
from keras import layers
from keras.utils import np_utils
import keras.backend as K

In [28]:
# 하이퍼퍼라미터
embed_size = 256
rnn_units = 4096

# 데이터 이름
data_name = 'Rachmaninov'
pickle_name = 'Rachmaninov_pickle'

In [29]:
with open('./preprocessing/data/{}_notes'.format(pickle_name), 'rb') as f:
  notes = pickle.load(f) #['G2', 'D3', 'B3', 'A3', 'B3', 'D3', 'B3', 'D3', 'G2',...]
with open('./preprocessing/data/{}_durations'.format(pickle_name), 'rb') as f:
  durations = pickle.load(f)

In [30]:
def get_distinct(elements):
    # Get all pitch names
    element_names = sorted(set(elements))
    n_elements = len(element_names)
    return (element_names, n_elements)

In [31]:
def create_lookups(element_names):
    # create dictionary to map notes and durations to integers
    element_to_int = dict((element, number) for number, element in enumerate(element_names))
    int_to_element = dict((number, element) for number, element in enumerate(element_names))

    return (element_to_int, int_to_element)

In [32]:
# 고유한 음표와 박자 얻어오기
note_names, n_notes = get_distinct(notes)
duration_names, n_durations = get_distinct(durations)
distincts = [note_names, n_notes, duration_names, n_durations]

with open('./preprocessing/data/{}_distincts'.format(pickle_name), 'wb') as f:
    pickle.dump(distincts, f)

# 음표와 박자 룩업 딕셔너리 만들고 저장하기
note_to_int, int_to_note = create_lookups(note_names)
duration_to_int, int_to_duration = create_lookups(duration_names)
lookups = [note_to_int, int_to_note, duration_to_int, int_to_duration]

with open('./preprocessing/data/{}_lookups'.format(pickle_name), 'wb') as f:
    pickle.dump(lookups, f)

In [33]:
note_to_int

{'A1': 0,
 'A1.A2': 1,
 'A1.E2': 2,
 'A2': 3,
 'A2.A1': 4,
 'A2.A3': 5,
 'A2.C#3': 6,
 'A2.C#3.F3.B3': 7,
 'A2.C#5': 8,
 'A2.C3': 9,
 'A2.D2': 10,
 'A2.E3.A3': 11,
 'A2.F#3': 12,
 'A2.G3': 13,
 'A3': 14,
 'A3.A1': 15,
 'A3.A1.A2': 16,
 'A3.A2': 17,
 'A3.A2.A1': 18,
 'A3.A4': 19,
 'A3.A4.A1.A2': 20,
 'A3.A4.A2': 21,
 'A3.A4.F#4.C#4.F#3.C#3': 22,
 'A3.C#4': 23,
 'A3.C#4.A4.E3': 24,
 'A3.C#4.E4': 25,
 'A3.C#4.F4': 26,
 'A3.C4.A4.C2': 27,
 'A3.C4.D2': 28,
 'A3.C4.D4.A4.D3.F2': 29,
 'A3.C4.E-4.A4': 30,
 'A3.C4.E-4.A4.E-3.F#2': 31,
 'A3.C4.F#2': 32,
 'A3.C4.F#4.D3.D2': 33,
 'A3.C4.F4.A4.F3.C3.F2': 34,
 'A3.D3': 35,
 'A3.D4': 36,
 'A3.D4.F#4.C2.C3': 37,
 'A3.D4.F3': 38,
 'A3.D4.F4': 39,
 'A3.D4.F4.A4': 40,
 'A3.D4.F4.A4.A2.D2': 41,
 'A3.D4.F4.A4.D2.A2': 42,
 'A3.D4.F4.D2.D3': 43,
 'A3.E-4': 44,
 'A3.E4.A4.C#4.E3': 45,
 'A3.E4.C#4.A4': 46,
 'A3.F#4.A4.C#3.F#3.C#4': 47,
 'A3.F#4.E-4': 48,
 'A3.F3.A2': 49,
 'A3.F3.D4.A2.D2': 50,
 'A3.F3.D4.F4.D2.A2': 51,
 'A3.F3.F4.C3.F2': 52,
 'A3.F4.A4.B-2': 5

In [34]:
duration_to_int

{0: 0,
 0.25: 1,
 Fraction(1, 3): 2,
 0.5: 3,
 Fraction(2, 3): 4,
 0.75: 5,
 1.0: 6,
 1.25: 7,
 Fraction(4, 3): 8,
 1.5: 9,
 Fraction(5, 3): 10,
 1.75: 11,
 2.0: 12,
 2.25: 13,
 Fraction(7, 3): 14,
 2.5: 15,
 Fraction(8, 3): 16,
 2.75: 17,
 3.0: 18,
 4.0: 19,
 4.25: 20,
 4.5: 21,
 5.0: 22,
 6.0: 23,
 8.0: 24,
 10.0: 25,
 11.0: 26,
 12.0: 27}

In [35]:
def prepare_sequences(notes, durations, lookups, distincts):
    """ Prepare the sequences used to train the Neural Network """
    seq_len = 100

    note_to_int, int_to_note, duration_to_int, int_to_duration = lookups
    note_names, n_notes, duration_names, n_durations = distincts

    notes_network_input = []
    notes_network_output = []
    durations_network_input = []
    durations_network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - seq_len, 1):
        notes_sequence_in = notes[i:i + seq_len]
        notes_sequence_out = notes[i + seq_len]
        notes_network_input.append([note_to_int[char] for char in notes_sequence_in])
        notes_network_output.append(note_to_int[notes_sequence_out])

        durations_sequence_in = durations[i:i + seq_len]
        durations_sequence_out = durations[i + seq_len]
        durations_network_input.append([duration_to_int[char] for char in durations_sequence_in])
        durations_network_output.append(duration_to_int[durations_sequence_out])

    n_patterns = len(notes_network_input)

    # reshape the input into a format compatible with LSTM layers
    notes_network_input = np.reshape(notes_network_input, (n_patterns, seq_len))
    durations_network_input = np.reshape(durations_network_input, (n_patterns, seq_len))
    network_input = [notes_network_input, durations_network_input]

    notes_network_output = np_utils.to_categorical(notes_network_output, num_classes=n_notes)
    durations_network_output = np_utils.to_categorical(durations_network_output, num_classes=n_durations)
    network_output = [notes_network_output, durations_network_output]

    return (network_input, network_output)

In [36]:
network_input, network_output = prepare_sequences(notes, durations, lookups, distincts)

In [37]:
notes_in = layers.Input(shape = (None, ), name = 'notes_in')
durations_in = layers.Input(shape = (None, ), name = 'durations_in')

x1 = layers.Embedding(n_notes, embed_size)(notes_in)
x2 = layers.Embedding(n_durations, embed_size)(durations_in)

x = layers.Concatenate()([x1, x2])

x = layers.LSTM(
    rnn_units,
    return_sequences = True,
)(x)

x = layers.LSTM(
    rnn_units,
    return_sequences = True
)(x)

e = layers.Dense(1, activation = 'tanh')(x)
e = layers.Reshape([-1])(e)
alpha = layers.Softmax()(e)

alpha_repeated = layers.RepeatVector(rnn_units)(alpha)

c = layers.Permute([2, 1])(alpha_repeated)
c = layers.Multiply()([x, c])
c = layers.Lambda(lambda xin: K.sum(xin, axis = 1), output_shape = (rnn_units,))(c)

notes_out = layers.Dense(n_notes, activation = 'softmax', name = 'pitch')(c)
durations_out = layers.Dense(n_durations, activation = 'softmax', name = 'duration')(c)

model = models.Model([notes_in, durations_in], [notes_out, durations_out], name = 'model')
att_model = models.Model([notes_in, durations_in], alpha)

opti = optimizers.RMSprop(lr = 0.001)
model.compile(loss = ['categorical_crossentropy', 'categorical_crossentropy'],
              optimizer = opti)

In [38]:
import time
import os
import sys
from music21 import *

import matplotlib.pyplot as plt

In [39]:
store_folder = './preprocessing/data'
pickle_name = 'Rachmaninov_pickle'

with open(os.path.join(store_folder, '{}_distincts'.format(pickle_name)), 'rb') as filepath:
    distincts = pickle.load(filepath)
    note_names, n_notes, duration_names, n_durations = distincts

with open(os.path.join(store_folder, '{}_lookups'.format(pickle_name)), 'rb') as filepath:
    lookups = pickle.load(filepath)
    note_to_int, int_to_note, duration_to_int, int_to_duration = lookups

In [40]:
model.load_weights('./weights/Rebuild_LSTM/Rachmaninov/weights.h5')
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
notes_in (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
durations_in (InputLayer)       [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 256)    554240      notes_in[0][0]                   
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 256)    7168        durations_in[0][0]               
______________________________________________________________________________________________

In [41]:
note_to_int['START']

2164

In [42]:
# 예측용 파라미터
notes_temp=0.5
duration_temp = 0.5

max_extra_notes = 600
max_seq_len = 100
seq_len = 100

# notes = ['START', 'D3', 'D3', 'E3', 'D3', 'G3', 'F#3','D3', 'D3', 'E3', 'D3', 'G3', 'F#3','D3', 'D3', 'E3', 'D3', 'G3', 'F#3','D3', 'D3', 'E3', 'D3', 'G3', 'F#3']
# durations = [0, 0.75, 0.25, 1, 1, 1, 2, 0.75, 0.25, 1, 1, 1, 2, 0.75, 0.25, 1, 1, 1, 2, 0.75, 0.25, 1, 1, 1, 2]


# notes = ['START', 'F#3', 'G#3', 'F#3', 'E3', 'F#3', 'G#3', 'F#3', 'E3', 'F#3', 'G#3', 'F#3', 'E3','F#3', 'G#3', 'F#3', 'E3', 'F#3', 'G#3', 'F#3', 'E3', 'F#3', 'G#3', 'F#3', 'E3']
# durations = [0, 0.75, 0.25, 1, 1, 1, 2, 0.75, 0.25, 1, 1, 1, 2, 0.75, 0.25, 1, 1, 1, 2, 0.75, 0.25, 1, 1, 1, 2]


notes = ['START']
durations = [0]

if seq_len is not None:
    notes = ['START'] * (seq_len - len(notes)) + notes
    durations = [0] * (seq_len - len(durations)) + durations


sequence_length = len(notes)

In [43]:
def sample_with_temp(preds, temperature):

    if temperature == 0:
        return np.argmax(preds)
    else:
        preds = np.log(preds) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        return np.random.choice(len(preds), p=preds)

In [ ]:
use_attention = True

prediction_output = []
notes_input_sequence = []
durations_input_sequence = []

overall_preds = []

for n, d in zip(notes,durations):
    note_int = note_to_int[n]
    duration_int = duration_to_int[d]
    
    notes_input_sequence.append(note_int)
    durations_input_sequence.append(duration_int)
    
    prediction_output.append([n, d])
    
    if n != 'START':
        midi_note = note.Note(n)

        new_note = np.zeros(128)
        new_note[midi_note.pitch.midi] = 1
        overall_preds.append(new_note)


att_matrix = np.zeros(shape = (max_extra_notes+sequence_length, max_extra_notes))

for note_index in range(max_extra_notes):

    prediction_input = [
        np.array([notes_input_sequence])
        , np.array([durations_input_sequence])
       ]
    # print('\ninput============', prediction_input, prediction_input[0].shape, prediction_input[1].shape, sep = '\n')
    notes_prediction, durations_prediction = model.predict(prediction_input, verbose=0)
    # print('\npredict==========', notes_prediction, durations_prediction, len(notes_prediction[0]), len(durations_prediction[0]), sep = '\n')
    if use_attention:
        att_prediction = att_model.predict(prediction_input, verbose=0)[0]
        att_matrix[(note_index-len(att_prediction)+sequence_length):(note_index+sequence_length), note_index] = att_prediction
    
    new_note = np.zeros(128)
    
    for idx, n_i in enumerate(notes_prediction[0]):
        try:
            note_name = int_to_note[idx]
            midi_note = note.Note(note_name)
            new_note[midi_note.pitch.midi] = n_i
            
        except:
            pass
        
    overall_preds.append(new_note)
            
    
    i1 = sample_with_temp(notes_prediction[0], notes_temp)
    i2 = sample_with_temp(durations_prediction[0], duration_temp)
    

    note_result = int_to_note[i1]
    duration_result = int_to_duration[i2]
    
    prediction_output.append([note_result, duration_result])

    notes_input_sequence.append(i1)
    durations_input_sequence.append(i2)
    
    if len(notes_input_sequence) > max_seq_len:
        notes_input_sequence = notes_input_sequence[1:]
        durations_input_sequence = durations_input_sequence[1:]
        
    print(note_result, duration_result)
        
    if note_result == 'START':
        break

overall_preds = np.transpose(np.array(overall_preds)) 
print('Generated sequence of {} notes'.format(len(prediction_output)))

C1 1.5
A4.E4.C#4.A3.E3.A2 0
C2 0
G2 1.0
G2 1.0
G2 1.0
E3 1.0
C3 0.5
C3 0.25
G2 0.25
C3 0.75
G2 1.0
C3 0.25
B-2 0


In [ ]:
overall_preds[35:70,:]

In [ ]:
fig, ax = plt.subplots(figsize=(20,20))
ax.set_yticks([int(j) for j in range(35, 70)])

plt.imshow(overall_preds[35 : 70, :], origin="lower", cmap='coolwarm', vmin = -0.5, vmax = 0.5,
           extent=[0, 35, 35,70]) # extent=[0, max_extra_notes, 35,70]
plt.show()

In [ ]:
output_folder = './output/Rebuild_LSTM/Rachmaninov'

midi_stream = stream.Stream()

# 모델이 생성한 값을 기반으로 악보와 화음 객체 만들기
for pattern in prediction_output:
    note_pattern, duration_pattern = pattern
    # 패턴이 화음일 경우
    if ('.' in note_pattern):
        notes_in_chord = note_pattern.split('.')
        chord_notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(current_note)
            new_note.duration = duration.Duration(duration_pattern)
            new_note.storedInstrument = instrument.Violoncello()
            chord_notes.append(new_note)
        new_chord = chord.Chord(chord_notes)
        midi_stream.append(new_chord)
    elif note_pattern == 'rest':
    # 패턴이 쉼표일 경우
        new_note = note.Rest()
        new_note.duration = duration.Duration(duration_pattern)
        new_note.storedInstrument = instrument.Violoncello()
        midi_stream.append(new_note)
    elif note_pattern != 'START':
    # 패턴이 하나의 음표일 경우
        new_note = note.Note(note_pattern)
        new_note.duration = duration.Duration(duration_pattern)
        new_note.storedInstrument = instrument.Violoncello()
        midi_stream.append(new_note)



midi_stream = midi_stream.chordify()
timestr = time.strftime("%Y%m%d-%H%M%S")
midi_stream.write('midi', fp=os.path.join(output_folder, 'output-' + timestr + '.mid'))

In [ ]:
## 어텐션 그래프
if use_attention:
    fig, ax = plt.subplots(figsize=(30,30))

    im = ax.imshow(att_matrix[(seq_len-2):,], cmap='coolwarm', interpolation='nearest')


    # Minor ticks
    ax.set_xticks(np.arange(-.5, len(prediction_output)- seq_len, 1), minor=True);
    ax.set_yticks(np.arange(-.5, len(prediction_output)- seq_len, 1), minor=True);

    # Gridlines based on minor ticks
    ax.grid(which='minor', color='black', linestyle='-', linewidth=1)
    
    
    # We want to show all ticks...
    ax.set_xticks(np.arange(len(prediction_output) - seq_len))
    ax.set_yticks(np.arange(len(prediction_output)- seq_len+2))
    # ... and label them with the respective list entries
    ax.set_xticklabels([n[0] for n in prediction_output[(seq_len):]])
    ax.set_yticklabels([n[0] for n in prediction_output[(seq_len - 2):]])

    # ax.grid(color='black', linestyle='-', linewidth=1)

    ax.xaxis.tick_top()


    
    plt.setp(ax.get_xticklabels(), rotation=90, ha="left", va = "center",
             rotation_mode="anchor")

    plt.show()